# A sample Analysis to Perform Spam Mail detection classificaton

. This is a simple mail classfication problem done using various Algotithms

### The Source of the data was from Kaggle
### The data Was classified using the following allgorithms.
1. MultinomialNb - A naivebayes algorithm that uses baysian approach to claassify
2. XGBClassifier - A boosting algorithm build using enseble (tree ones) ones
3. RandomForestClassifier - Which is an ensembe buld from a group od decision trees


### THey Were used to compare perfomance both in training and test sets

In [47]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
#read the data
mails = pd.read_csv("emails.csv")

In [3]:
mails.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [4]:
mails.tail()

,text,spam
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0
5727,Subject: news : aurora 5 . 2 update aurora ve...,0


In [5]:
#get rows and columns size
mails.shape

(5728, 2)

In [6]:
# get null values
mails.isnull().sum()

text    0
spam    0
dtype: int64

In [7]:
#check if there are duplicates
mails.duplicated().sum()

33

In [8]:
#drop duplicates
mails.drop_duplicates(inplace = True)

In [9]:
#check the new rows and columns of the data
#since duplicated rows were removed
mails.shape

(5695, 2)

In [10]:
#the messages seems to have subject key word and re in some
#after colon
#let check the first 5 columns
mails.text.str.split(":").str.get(0)[:5]

0    Subject
1    Subject
2    Subject
3    Subject
4    Subject
Name: text, dtype: object

In [11]:
#lets start cleaning our data
#first we will remove the the keywords
#eg remove 
#1 subject keyword
#2 re keyword


def remove_keywords(text):
    text = text.replace("Subject" , "")
    text = text.replace("Subject" , "")
    text = text.replace("re" , "")
    text = text.replace("Subject" , "")
    
    return text
    

In [12]:
# $lets check if it is working with 5 instances first before for all dat
mails['text'].tail().apply(remove_keywords)

5723    :  : search and development charges to gpg  he...
5724    :  : ceipts from visit  jim ,  thanks again fo...
5725    :  : enron case study update  wow ! all on the...
5726    :  : intest  david ,  please , call shirley cn...
5727    : news : aurora 5 . 2 update  aurora version 5...
Name: text, dtype: object

In [13]:
#lets check the way the first 4 rows looks
for each in mails['text'].head(4):
    print("\n\n")
    print(each)




Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .  we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader : it isguite ciear that  without good products , effective business organization and practicable aim it  will be hotat nowadays market ; but we do promise that your marketing efforts  will become much more effective . here is the list of clear  benefits : creativeness : hand - made , original logos , specially done  to reflect your distinctive company image . convenience : logo and stationery  are provided in all formats ; easy - to - use content management system letsyou  change your website content and even its structure . promptness : you  will see logo drafts within three business days . affordability : your  

In [14]:
#observations
#we need to remove
#1 Special strings and characters
#2 Digits
#3 phone numbers
#4 stopwords

In [15]:
#check if we have urls
pattern = r"(?P<protocol>https?)://(?P<domain>[\w\.\-]+)/?(?P<path>.*)"
urls = mails['text'].str.extract(pattern)

In [16]:
urls.sum()
#thre are no urls

protocol    0.0
domain      0.0
path        0.0
dtype: float64

In [17]:
#removing emails keywords
#we will replace with ""
import re

pattern = r"e[-\s]?mail"
email_uniform = mails['text'].str.replace(pattern , "email" , flags=re.I)

In [18]:
#working with numbers
pattern =r"([0-9]+)"
nums = mails["text"].str.extractall(pattern)

In [19]:
# lets now clean it
#1 Special strings and characters
#2 Digits
#3 phone numbers
#4 stopwords
import string
import re
x = string.punctuation
pattern =r"([0-9]+)"
def clearAll(text):
    text = text.replace(string.punctuation , "")
#     lack_special_chars = [x for x in text if x not in string.punctuation]

#     join the word using spaces
#     lack_special_chars = ''.join(lack_special_chars)
    
    return text
    
    

In [20]:
mails.head(4).apply(clearAll)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1


In [21]:
# lets apply the functions
mails['text'] = mails['text'].apply(remove_keywords)
mails['text'] = mails['text'].apply(clearAll)

In [22]:
mails['text'].tail()

5723    :  : search and development charges to gpg  he...
5724    :  : ceipts from visit  jim ,  thanks again fo...
5725    :  : enron case study update  wow ! all on the...
5726    :  : intest  david ,  please , call shirley cn...
5727    : news : aurora 5 . 2 update  aurora version 5...
Name: text, dtype: object

In [23]:
# remove any non chars
for c in range(mails.shape[0]):
    mails['text'].iloc[c] = [x for x in mails['text'].iloc[c] if x not in string.punctuation]
    mails['text'].iloc[c] =''.join(mails['text'].iloc[c])

In [24]:
mails['text'].tail()

5723       search and development charges to gpg  he i...
5724       ceipts from visit  jim   thanks again for t...
5725       enron case study update  wow  all on the sa...
5726       intest  david   please  call shirley cnshaw...
5727     news  aurora 5  2 update  aurora version 5  2...
Name: text, dtype: object

In [25]:
# pattern = r"([0-9]+)"
#remove the digits
number = ["1" , "2","3" ,'4' ,'5','6','7','8','9' ,"0"]
for c in  range(mails.shape[0]):
    mails['text'].iloc[c] = [x for x in mails['text'].iloc[c] if x not in number]
    mails['text'].iloc[c] =''.join(mails['text'].iloc[c])
    
    
    

In [26]:
#lets remove the stop words
import nltk
#using stop words
# nltk.download('stopwords')
from nltk.corpus import stopwords

In [27]:
#remove stopwords from the text column

for c in  range(mails.shape[0]):
    mails['text'].iloc[c] = [x for x in mails['text'].iloc[c] if x not in stopwords.words('english')]
    mails['text'].iloc[c] =''.join(mails['text'].iloc[c])

In [28]:
#check new data
mails.head()

,text,spam
0,nurll rble ur crpre en l ll hr cllec cpn ...,1
1,he ck rng gunlnger fnn errll bu uz n clz ne...,1
2,unbelevble new he e e wnng hw u h hewner ...,1
3,clr prnng pecl que nl nfrn nw clck he clc...,1
4,n hve ne ge fw c fr he fw cpbl n g...,1


In [29]:
#do vectorization and covert it to matrix
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()

text = vec.fit_transform(mails['text'])


In [30]:
#get the labels
y = mails['spam']
text.shape


(5695, 14618)

In [31]:
#do splitliting the data 80 to 20
from sklearn.model_selection import train_test_split
x_train , x_test,y_train  , y_test = train_test_split(text , y , test_size = 0.2 , random_state = 0)

In [32]:
#training using naibayes classifier(multinomial)
from sklearn.naive_bayes import MultinomialNB
clf1 = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


In [33]:
#cross validation for naive bayes algo
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf1 , X =text , y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[0.98596491 0.98947368 0.97894737 0.97719298 0.99473684 0.98421053
 0.9754386  0.98418278 0.98943662 0.99119718]
Mean for cross val is 0.985078149189901
std for the cross val is 0.006033854752178476


In [34]:
#do training baesd on multinomialnb
clf1.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#  Printing Performance for MultiNomialNb Classifier

### Checking on Training ste

In [35]:
#checking score based on train set
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred = clf1.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred)}")

Classification Repport
                precision    recall  f1-score   support

           0       1.00      0.99      0.99      3457
           1       0.97      1.00      0.98      1099

    accuracy                           0.99      4556
   macro avg       0.98      0.99      0.99      4556
weighted avg       0.99      0.99      0.99      4556




Confudion Matrix  
 [[3424   33]
 [   4 1095]]



Accuracy score   0.9918788410886743


## Checking score on the testing datasets

In [36]:
#checking score based on set test
pred = clf1.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred)}")

Classification Repport
                precision    recall  f1-score   support

           0       0.99      0.98      0.99       870
           1       0.95      0.98      0.96       269

    accuracy                           0.98      1139
   macro avg       0.97      0.98      0.98      1139
weighted avg       0.98      0.98      0.98      1139




Confudion Matrix  
 [[855  15]
 [  5 264]]



Accuracy score   0.9824407374890255


## Trying other models 

In [37]:
#randomclassifier model
from sklearn.ensemble import RandomForestClassifier

clf2 = RandomForestClassifier()


In [38]:
#cross validation for random classifeir
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf2 , X =text , y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[0.93333333 0.94912281 0.95087719 0.94385965 0.94385965 0.95087719
 0.94035088 0.92970123 0.92957746 0.96302817]
Mean for cross val is 0.9434587565785673
std for the cross val is 0.010097396202320603


In [39]:
clf2.fit(x_train ,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [40]:
#checking score based on train set
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred2 = clf2.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred2)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred2)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred2)}")

Classification Repport
                precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       1.00      0.99      0.99      1099

    accuracy                           1.00      4556
   macro avg       1.00      0.99      1.00      4556
weighted avg       1.00      1.00      1.00      4556




Confudion Matrix  
 [[3457    0]
 [  12 1087]]



Accuracy score   0.9973661106233538


#  Printing Performance for XGBClassifier

In [41]:
#checking score based on set test
pred1 = clf2.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred1)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred1)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred1)}")

Classification Repport
                precision    recall  f1-score   support

           0       0.91      1.00      0.95       870
           1       0.98      0.68      0.81       269

    accuracy                           0.92      1139
   macro avg       0.95      0.84      0.88      1139
weighted avg       0.93      0.92      0.92      1139




Confudion Matrix  
 [[867   3]
 [ 85 184]]



Accuracy score   0.922739244951712


In [42]:
#lets check for boosting algo
from xgboost import XGBClassifier
clf3 = XGBClassifier()

In [43]:
#cross validation
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf3 , X =text , y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[0.98070175 0.9877193  0.98245614 0.97719298 0.9754386  0.98421053
 0.98421053 0.98418278 0.97711268 0.97359155]
Mean for cross val is 0.9806816826714921
std for the cross val is 0.004387585353333118


In [44]:
clf3.fit(x_train , y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [45]:
#checking score based on train set
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred3 = clf3.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred3)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred3)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred3)}")

Classification Repport
                precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       1.00      1.00      1.00      1099

    accuracy                           1.00      4556
   macro avg       1.00      1.00      1.00      4556
weighted avg       1.00      1.00      1.00      4556




Confudion Matrix  
 [[3457    0]
 [   2 1097]]



Accuracy score   0.9995610184372257


In [46]:
#checking score based on set test
pred3 = clf3.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred3)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred3)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred3)}")

Classification Repport
                precision    recall  f1-score   support

           0       0.98      0.99      0.99       870
           1       0.97      0.94      0.95       269

    accuracy                           0.98      1139
   macro avg       0.98      0.96      0.97      1139
weighted avg       0.98      0.98      0.98      1139




Confudion Matrix  
 [[863   7]
 [ 17 252]]



Accuracy score   0.9789288849868305
